量化obj文件，将new1中的obj文件量化到new2中，效果不好，最好不用

In [ ]:
import os


def quantize_value(value, min_val, max_val):
    """
    量化单个数值到 0 到 64 的范围
    :param value: 待量化的数值
    :param min_val: 当前数据的最小值
    :param max_val: 当前数据的最大值
    :return: 量化后的数值
    """
    # 处理 max_val 和 min_val 相等的情况
    if max_val == min_val:
        return 0
    # 先将数值归一化到 0 到 1 的范围
    normalized = (float(value) - min_val) / (max_val - min_val)
    # 再将归一化后的值映射到 0 到 64 的范围
    quantized = int(normalized * 64)
    return quantized


def process_obj_file(input_file_path, output_file_path):
    # print(f"开始处理文件: {input_file_path}")
    v_values = []
    vt_values = []
    # 第一步：读取文件，收集 v 行和 vt 行的所有数值
    with open(input_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith('v '):
                parts = line.split()[1:]
                v_values.extend([float(val) for val in parts])
            elif line.startswith('vt '):
                parts = line.split()[1:]
                vt_values.extend([float(val) for val in parts])

    # 第二步：计算 v 行和 vt 行数值的最小值和最大值
    if v_values:
        v_min = min(v_values)
        v_max = max(v_values)
    else:
        v_min = v_max = 0
    if vt_values:
        vt_min = min(vt_values)
        vt_max = max(vt_values)
    else:
        vt_min = vt_max = 0
    # print(f"v 行数值范围: [{v_min}, {v_max}]")
    # print(f"vt 行数值范围: [{vt_min}, {vt_max}]")

    new_lines = []
    # 第三步：对每一行进行处理
    for line in lines:
        if line.startswith('f '):
            parts = line.split()
            new_parts = ['f']
            for part in parts[1:]:
                # 提取 '/' 之前的数字
                vertex_index = part.split('/')[0]
                new_parts.append(vertex_index)
            new_line = ' '.join(new_parts) + '\n'
            new_lines.append(new_line)
        elif line.startswith('v '):
            parts = line.split()
            new_parts = [parts[0]]
            for val in parts[1:]:
                # 对 v 行的数值进行量化
                new_val = quantize_value(val, v_min, v_max)
                new_parts.append(str(new_val))
            new_line = ' '.join(new_parts) + '\n'
            new_lines.append(new_line)
        elif line.startswith('vt '):
            parts = line.split()
            new_parts = [parts[0]]
            for val in parts[1:]:
                # 对 vt 行的数值进行量化
                new_val = quantize_value(val, vt_min, vt_max)
                new_parts.append(str(new_val))
            new_line = ' '.join(new_parts) + '\n'
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # 第四步：将处理后的行写入新文件
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)
    # print(f"文件处理完成，保存到: {output_file_path}")


def process_folder(input_folder, output_folder, max_count=None):
    print(f"开始处理文件夹: {input_folder}")
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 初始化计数器
    processed_count = 1
    skipped_count = 1

    # 遍历输入文件夹下的所有文件和子文件夹
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.obj'):
                if max_count is not None and processed_count + skipped_count > max_count:
                    break
                input_file_path = os.path.join(root, file)
                # 保持文件名不变，构建输出文件路径
                output_file_path = os.path.join(output_folder, file)

                if os.path.exists(output_file_path):
                    print(f"文件 {output_file_path} 已存在，跳过处理。[{skipped_count}]")
                    skipped_count += 1
                    continue

                # print(f"[{processed_count + skipped_count}] 开始处理文件: {input_file_path}")
                process_obj_file(input_file_path, output_file_path)
                # print(f"[{processed_count + skipped_count}] 文件 {input_file_path} 处理完成，保存到: {output_file_path}")
                processed_count += 1
        if max_count is not None and processed_count + skipped_count > max_count:
            break
    # print(f"文件夹 {input_folder} 处理完成，结果保存到: {output_folder}，共处理 {processed_count - 1} 个文件，跳过 {skipped_count} 个文件。")


# 指定输入和输出文件夹路径
input_folder = ''
output_folder = ''
# 指定最大处理文件数量，若为 None 则处理所有文件
max_count = None
process_folder(input_folder, output_folder, max_count)
print("处理完成")